### Imports

In [ ]:
from datetime import datetime
import time

from contracts_lib_py.account import Account
from common_utils_py.agreements.service_types import ServiceTypesIndices


from nevermined_sdk_py import Config, Nevermined
from nevermined_sdk_py.nevermined.keeper import NeverminedKeeper as Keeper

from workshop import utils

CONSUMER_ADDRESS = "0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e"
CONSUMER_PASSWORD = "node0"
CONSUMER_KEYFILE = "../resources/accounts/consumer.json"
CONFIG_FILE = "../config.ini"

PROVIDER_ADDRESS = "0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0"
ASSET_COMPUTE_DID_1 = "did:nv:35ea2cf4fbce0ab1b120f2c330a33ee9e41478c7094cf8553caff32b7b7c51df"
ASSET_COMPUTE_DID_2 = "did:nv:eb046660d3e8a85efcfd0038e7636efa58d0a83488a47c89e110ef40dbc584b9"
ASSET_COORDINATOR_DID = "did:nv:347531a023342f93d2c9b9b55eabe2a961bffc53284d79e01f667428d0b540c0"

### Setup Nevermined and accounts

In [ ]:
nevermined = Nevermined(Config(CONFIG_FILE))
consumer_account = Account(CONSUMER_ADDRESS, CONSUMER_PASSWORD, CONSUMER_KEYFILE)

nevermined.accounts.request_tokens(consumer_account, 10)


### Publish algorithm

In [ ]:
metadata_algorithm = utils.metadata_algorithm_template()

metadata_algorithm["main"]["dateCreated"] = utils.date_now()
metadata_algorithm["main"]["files"][0]["url"] = "https://github.com/nevermined-io/hellodecentralization/raw/main/notebooks/federated_fraud_demo.ipynb"

metadata_algorithm["main"]["algorithm"]["entrypoint"] = (
    "pip install jupyter pandas papermill scikit-learn xain-sdk && "
    "papermill --stdout-file - federated_fraud_demo.ipynb $NEVERMINED_OUTPUTS_PATH/federated_fraud_demo_output.ipynb"
)

metadata_algorithm["main"]["algorithm"]["requirements"]["container"]["image"] = "python"
metadata_algorithm["main"]["algorithm"]["requirements"]["container"]["tag"] = "3.8-slim-buster"

utils.print_json(metadata_algorithm)

ddo_algorithm = nevermined.assets.create(metadata_algorithm, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Algorithm DID: {ddo_algorithm.did}")

### Publish the workflows:

- Two for the compute to the data assets
- One for the coordinator service

In [ ]:
# Compute asset 1
metadata_workflow1 = utils.metadata_workflow_template()

metadata_workflow1["main"]["dateCreated"] = utils.date_now()
metadata_workflow1["main"]["workflow"]["stages"][0]["input"][0]["id"] = ASSET_COMPUTE_DID_1
metadata_workflow1["main"]["workflow"]["stages"][0]["transformation"]["id"] = ddo_algorithm.did

utils.print_json(metadata_workflow1)


ddo_workflow1 = nevermined.assets.create(metadata_workflow1, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow1.did}")

# Compute asset 2
metadata_workflow2 = utils.metadata_workflow_template()
metadata_workflow2["main"]["dateCreated"] = utils.date_now()
metadata_workflow2["main"]["workflow"]["stages"][0]["input"][0]["id"] = ASSET_COMPUTE_DID_2
metadata_workflow2["main"]["workflow"]["stages"][0]["transformation"]["id"] = ddo_algorithm.did

utils.print_json(metadata_workflow2)

ddo_workflow2 = nevermined.assets.create(metadata_workflow2, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow2.did}")

# Coordinator service
metadata_workflow_coordinator = utils.metadata_workflow_coordinator()
metadata_workflow_coordinator["main"]["dateCreated"] = utils.date_now()
metadata_workflow_coordinator["main"]["datePublished"] = utils.date_now()

utils.print_json(metadata_workflow_coordinator)


ddo_workflow_coordinator = nevermined.assets.create(metadata_workflow_coordinator, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow_coordinator.did}")

### Order computations:

- One for each data asset
- One for the coordinator service

In between we want for the blockchain to process the request.

In [ ]:
keeper = Keeper.get_instance()

service_agreement_id1 = nevermined.assets.order(ASSET_COMPUTE_DID_1, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id1}")
utils.wait_for_event(keeper, service_agreement_id1)

service_agreement_id2 = nevermined.assets.order(ASSET_COMPUTE_DID_2, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id2}")
utils.wait_for_event(keeper, service_agreement_id2)

service_agreement_id_coordinator = nevermined.assets.order(ASSET_COORDINATOR_DID, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id_coordinator}")
utils.wait_for_event(keeper, service_agreement_id_coordinator)


### Execute workflows

This orders the Nevermined to start the execution of the workflows:
- One for each data asset
- One for the coordinator service

In [ ]:
execution_id1 = nevermined.assets.execute(
    service_agreement_id1,
    ASSET_COMPUTE_DID_1,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow1.did
)
print(f"Execution ID: {execution_id1}")

execution_id2 = nevermined.assets.execute(
    service_agreement_id2,
    ASSET_COMPUTE_DID_2,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow2.did
)
print(f"Execution ID: {execution_id2}")

execution_id_coordinator = nevermined.assets.execute(
    service_agreement_id_coordinator,
    ASSET_COORDINATOR_DID,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow_coordinator.did
)
print(f"Execution ID: {execution_id_coordinator}")

### Wait for compute jobs to finish

In [ ]:
succeeded = set()
failed = set()
outputs = set()
while len(succeeded) < 3:
    for job_id in [execution_id1, execution_id2, execution_id_coordinator]:
        result = nevermined.assets.compute_status(service_agreement_id1, job_id, consumer_account)
        status = result["status"]
        print(f"{job_id}: {status}")

        if status == "Succeeded":
            succeeded.add(job_id)
            outputs.add(result["did"])
        elif status == "Failed":
            raise Exception("Some jobs failed")
    print()
    time.sleep(10)

for did in outputs:
    print(f"Output DID: {did}")

### Download the results

In [ ]:
for did in outputs:
    print(f"Downloading: {did}")
    nevermined.assets.download(did, ServiceTypesIndices.DEFAULT_ACCESS_INDEX, consumer_account, "./")